In [1]:
import numpy as np
from pyqgraf import qgraf, model
from feynml.interface.qgraf import style
import logging
logger = logging.getLogger("feynamp")
logger.setLevel(logging.DEBUG)


	Please cite the following papers if you use this code:

      [1] Automatic Feynman graph generation J. Comput. Phys. 105 (1993) 279--289 https://doi.org/10.1006/jcph.1993.1074

      [2] Abusing Qgraf Nucl. Instrum. Methods Phys. Res. A 559 (2006) 220--223 https://doi.org/10.1016/j.nima.2005.11.151

      [3] Feynman graph generation and propagator mixing, I Comput. Phys. Commun. 269 (2021) 108103 https://doi.org/10.1016/j.cpc.2021.108103

	


In [2]:
from feynmodel.interface.ufo import load_ufo_model
from feynmodel.interface.qgraf import feynmodel_to_qgraf
fm = load_ufo_model("ufo_sm")
qfm = feynmodel_to_qgraf(fm, True, False)

In [3]:
qgraf.install("3.6.5")
xml_string = qgraf.run("e_minus[p1], e_plus[p2]", "u[p3], u_bar[p4], g[p5]",
                       loops=0,
                       loop_momentum="l",
                       model = qfm, 
                       style=style,
                       debug=False)
#print(xml_string)

In [4]:
from xsdata.formats.dataclass.parsers import XmlParser
from pyfeyn2.feynmandiagram import FeynML

parser = XmlParser()
fml = parser.from_string(xml_string, FeynML)
len(fml.diagrams)

4

In [5]:
fd1,fd2 = np.array(fml.diagrams)[[f.has_pdgid(22) for f in fml.diagrams]]
fds = [fd1,fd2]
#fd1.legs = [fd1.legs[0],fd1.legs[1],fd1.legs[3],fd1.legs[2],fd1.legs[4]]
#fd2.legs = [fd2.legs[0],fd2.legs[1],fd2.legs[3],fd2.legs[2],fd2.legs[4]]

In [6]:
#fd2.with_rule("* {momentum-arrow: true}")

In [7]:
#fd2.conjugated().with_rule("* {momentum-arrow: true}")

In [8]:
from feynamp import amplitude as amp
am = amp.add([fd1],fm)
am

'(((-(ee*complex(0,1)))*(1)*(Gamma(MuInProp1,SpinIn2,SpinIn1))) * ((complex(0,1)*G)*(T(GluOut3,ColOut1,ColInProp2))*(Gamma(MuOut3,SpinOut1,SpinInProp2))) * (((2*ee*complex(0,1))/3.)*(Identity(ColOut2,ColOutProp2))*(Gamma(MuOutProp1,SpinOutProp2,SpinOut2))) * (u(SpinIn1,Mom_p1)) * (v(SpinIn2,Mom_p2)) * (VC(ColOut1,Mom_p3)*u_bar(SpinOut1,Mom_p3)) * (VC(ColOut2,Mom_p4)*v_bar(SpinOut2,Mom_p4)) * (VA(GluOut3,Mom_p5)*eps_star(MuOut3,PolOut3,Mom_p5)) * ((-1)*complex(0,1)*Metric(MuInProp1,MuOutProp1)*Denom(-Mom_p1-Mom_p2,0)) * (df(ColInProp2,ColOutProp2)*complex(0,1)*(P(Mu14,Mom_p3+Mom_p5)*Gamma(Mu14,SpinInProp2,SpinOutProp2) + 0*GammaId(SpinInProp2,SpinOutProp2))*Denom(Mom_p3+Mom_p5,0)))*1'

In [9]:
from feynamp import sympy as samp
samp.string_to_sympy(amp.feynman_diagram_to_string(fd1,fm))

0.666666666666667*I*G*ee**2*Denom(-Mom_p1 - Mom_p2, 0)*Denom(Mom_p3 + Mom_p5, 0)*Identity(ColOut2, ColOutProp2)*P(Mu21, Mom_p3 + Mom_p5)*T(GluOut3, ColOut1, ColInProp2)*VA(GluOut3, Mom_p5)*VC(ColOut1, Mom_p3)*VC(ColOut2, Mom_p4)*df(ColInProp2, ColOutProp2)*eps_star(MuOut3, PolOut3, Mom_p5)*gamma(Mu21, SpinInProp2, SpinOutProp2)*gamma(MuInProp1, SpinIn2, SpinIn1)*gamma(MuOut3, SpinOut1, SpinInProp2)*gamma(MuOutProp1, SpinOutProp2, SpinOut2)*metric(MuInProp1, MuOutProp1)*u(SpinIn1, Mom_p1)*u_bar(SpinOut1, Mom_p3)*v(SpinIn2, Mom_p2)*v_bar(SpinOut2, Mom_p4)

In [10]:
from feynamp.amplitude import square_parallel, multiply

s2 =square_parallel([fd1,fd2],fm,tag=False)
s2[1] = f"({s2[1]})"
s2[3] = f"({s2[3]})"
s2

['(((-(ee*complex(0,1)))*(1)*(Gamma(MuInProp1,SpinIn2,SpinIn1))) * ((complex(0,1)*G)*(T(GluOut3,ColOut1,ColInProp2))*(Gamma(MuOut3,SpinOut1,SpinInProp2))) * (((2*ee*complex(0,1))/3.)*(Identity(ColOut2,ColOutProp2))*(Gamma(MuOutProp1,SpinOutProp2,SpinOut2))) * (u(SpinIn1,Mom_p1)) * (v(SpinIn2,Mom_p2)) * (VC(ColOut1,Mom_p3)*u_bar(SpinOut1,Mom_p3)) * (VC(ColOut2,Mom_p4)*v_bar(SpinOut2,Mom_p4)) * (VA(GluOut3,Mom_p5)*eps_star(MuOut3,PolOut3,Mom_p5)) * ((-1)*complex(0,1)*Metric(MuInProp1,MuOutProp1)*Denom(-Mom_p1-Mom_p2,0)) * (df(ColInProp2,ColOutProp2)*complex(0,1)*(P(Mu28,Mom_p3+Mom_p5)*Gamma(Mu28,SpinInProp2,SpinOutProp2) + 0*GammaId(SpinInProp2,SpinOutProp2))*Denom(Mom_p3+Mom_p5,0)))*(((-(ee*complex(0,1)))*(1)*(Gamma(MuOutPropagator44,SpinLeg39,SpinLeg40))) * ((complex(0,1)*G)*(T(GluLeg43,ColOutPropagator45,ColLeg41))*(Gamma(MuLeg43,SpinOutPropagator45,SpinLeg41))) * (((2*ee*complex(0,1))/3.)*(Identity(ColInPropagator45,ColLeg42))*(Gamma(MuInPropagator44,SpinLeg42,SpinInPropagator45))) *

In [11]:
amp.feynman_diagram_to_string(fd1.conjugated(),fm)

'((-(ee*complex(0,1)))*(1)*(Gamma(MuOutPropagator78,SpinLeg73,SpinLeg74))) * ((complex(0,1)*G)*(T(GluLeg77,ColOutPropagator79,ColLeg75))*(Gamma(MuLeg77,SpinOutPropagator79,SpinLeg75))) * (((2*ee*complex(0,1))/3.)*(Identity(ColInPropagator79,ColLeg76))*(Gamma(MuInPropagator78,SpinLeg76,SpinInPropagator79))) * (u_bar(SpinLeg73,Mom_p1)) * (v_bar(SpinLeg74,Mom_p2)) * (VC(ColLeg75,Mom_p3)*u(SpinLeg75,Mom_p3)) * (VC(ColLeg76,Mom_p4)*v(SpinLeg76,Mom_p4)) * (VA(GluLeg77,Mom_p5)*eps(MuLeg77,PolLeg77,Mom_p5)) * ((-1)*complex(0,1)*Metric(MuInPropagator78,MuOutPropagator78)*Denom(-Mom_p1-Mom_p2,0)) * (df(ColInPropagator79,ColOutPropagator79)*complex(0,1)*(P(Mu86,Mom_p3+Mom_p5)*Gamma(Mu86,SpinInPropagator79,SpinOutPropagator79) + 0*GammaId(SpinInPropagator79,SpinOutPropagator79))*Denom(Mom_p3+Mom_p5,0))'

In [12]:
from feynamp.form.color import *
from feynamp.form.lorentz import *
from feynamp.form.momentum import *
fs = ""
fs += get_polarisation_sums(fds,fm)
fs += get_gammas()
fs += get_color()
fs += get_kinematics()
fs += get_onshell(fds,fm)
fs += get_mandelstamm_2_to_3(fds,fm)
rs =apply_parallel(s2,fs)
rs =" + ".join([f"({r})" for r in rs])
print(rs)

(32/9*Den(Mom_p1.Mom_p1+2*Mom_p1.Mom_p2+Mom_p2.Mom_p2)^2*Den(Mom_p3.Mom_p3+2*Mom_p3.Mom_p5+Mom_p5.Mom_p5)^2*G^2*ee^4*mss35*mst15*mst24+32/9*Den(Mom_p1.Mom_p1+2*Mom_p1.Mom_p2+Mom_p2.Mom_p2)^2*Den(Mom_p3.Mom_p3+2*Mom_p3.Mom_p5+Mom_p5.Mom_p5)^2*G^2*ee^4*mss35*mst14*mst25-32/9*Den(Mom_p1.Mom_p1+2*Mom_p1.Mom_p2+Mom_p2.Mom_p2)^2*Den(Mom_p3.Mom_p3+2*Mom_p3.Mom_p5+Mom_p5.Mom_p5)^2*G^2*ee^4*Mass_Me^2*mss35*mst25-32/9*Den(Mom_p1.Mom_p1+2*Mom_p1.Mom_p2+Mom_p2.Mom_p2)^2*Den(Mom_p3.Mom_p3+2*Mom_p3.Mom_p5+Mom_p5.Mom_p5)^2*G^2*ee^4*Mass_Me^2*mss35*mst24-32/9*Den(Mom_p1.Mom_p1+2*Mom_p1.Mom_p2+Mom_p2.Mom_p2)^2*Den(Mom_p3.Mom_p3+2*Mom_p3.Mom_p5+Mom_p5.Mom_p5)^2*G^2*ee^4*Mass_Me^2*mss35*mst15-32/9*Den(Mom_p1.Mom_p1+2*Mom_p1.Mom_p2+Mom_p2.Mom_p2)^2*Den(Mom_p3.Mom_p3+2*Mom_p3.Mom_p5+Mom_p5.Mom_p5)^2*G^2*ee^4*Mass_Me^2*mss35*mst14+64/9*Den(Mom_p1.Mom_p1+2*Mom_p1.Mom_p2+Mom_p2.Mom_p2)^2*Den(Mom_p3.Mom_p3+2*Mom_p3.Mom_p5+Mom_p5.Mom_p5)^2*G^2*ee^4*Mass_Me^4*mss35+64/9*Den(Mom_p1.Mom_p1+2*Mom_p1.Mom_p2+Mom_p2.M

In [13]:
rr =apply_den(rs, get_mandelstamm_2_to_3(fds,fm) + get_onshell(fds,fm))

In [14]:
rr

'(32/9*1/(mss12)^2*1/(mss35)^2*G^2*ee^4*mss35*mst15*mst24+32/9*1/(mss12)^2*1/(mss35)^2*G^2*ee^4*mss35*mst14*mst25-32/9*1/(mss12)^2*1/(mss35)^2*G^2*ee^4*Mass_Me^2*mss35*mst25-32/9*1/(mss12)^2*1/(mss35)^2*G^2*ee^4*Mass_Me^2*mss35*mst24-32/9*1/(mss12)^2*1/(mss35)^2*G^2*ee^4*Mass_Me^2*mss35*mst15-32/9*1/(mss12)^2*1/(mss35)^2*G^2*ee^4*Mass_Me^2*mss35*mst14+64/9*1/(mss12)^2*1/(mss35)^2*G^2*ee^4*Mass_Me^4*mss35+64/9*1/(mss12)^2*1/(mss35)^2*G^2*ee^4*Mass_Me^4*mss35*mss45-32/9*1/(mss12)^2*1/(mss35)^2*G^2*Nc^2*ee^4*mss35*mst15*mst24-32/9*1/(mss12)^2*1/(mss35)^2*G^2*Nc^2*ee^4*mss35*mst14*mst25+32/9*1/(mss12)^2*1/(mss35)^2*G^2*Nc^2*ee^4*Mass_Me^2*mss35*mst25+32/9*1/(mss12)^2*1/(mss35)^2*G^2*Nc^2*ee^4*Mass_Me^2*mss35*mst24+32/9*1/(mss12)^2*1/(mss35)^2*G^2*Nc^2*ee^4*Mass_Me^2*mss35*mst15+32/9*1/(mss12)^2*1/(mss35)^2*G^2*Nc^2*ee^4*Mass_Me^2*mss35*mst14-64/9*1/(mss12)^2*1/(mss35)^2*G^2*Nc^2*ee^4*Mass_Me^4*mss35-64/9*1/(mss12)^2*1/(mss35)^2*G^2*Nc^2*ee^4*Mass_Me^4*mss35*mss45) + (16/9*1/(mss12)^2*1/(ms

In [15]:
from sympy.parsing.sympy_parser import parse_expr
from sympy import *
ret =simplify(parse_expr(rr.replace("Mom_","").replace(".","_").replace("^","**")
                    .replace("Mass_Me","0")
                   .replace("mss","s")
                   .replace("msu","u")
                   .replace("mst","t")))
ret.subs("Nc","3").subs("Cf","4/3").subs("s34","-t13-t23-s35").simplify()

256*G**2*ee**4*(-s35*(t13*t24 + t13*t25 + t14*t23 - 2*t14*t24 + t15*t23) - s45*(-2*t13*t23 + t13*t24 + t14*t23 + t14*t25 + t15*t24) + (s35 + t13 + t23)*(2*t13*t24 + t13*t25 + 2*t14*t23 + t14*t25 + t15*t23 + t15*t24))/(9*s12**2*s35*s45)

In [16]:
# here we use the tags to set the right relative sign
ret = simplify(ret.subs("Nc","3").subs("Cf","4/3")
               .subs("s35","-t13-t23-s34")
               .subs("t15","-s12-t13-t14")
               .subs("t25","-s12-t23-t24")
               .subs("s45","-t14-s34-t24"))
ret

256*G**2*ee**4*(-s34*(2*t13*t24 - t13*(s12 + t23 + t24) + 2*t14*t23 - t14*(s12 + t23 + t24) - t23*(s12 + t13 + t14) - t24*(s12 + t13 + t14)) + (s34 + t13 + t23)*(t13*t24 - t13*(s12 + t23 + t24) + t14*t23 - 2*t14*t24 - t23*(s12 + t13 + t14)) + (s34 + t14 + t24)*(-2*t13*t23 + t13*t24 + t14*t23 - t14*(s12 + t23 + t24) - t24*(s12 + t13 + t14)))/(9*s12**2*(s34 + t13 + t23)*(s34 + t14 + t24))

In [22]:
sret = ret.subs({
          "t13" : "-2*s12*x_q/4*(1-cos(theta))",
          "t23" : "-2*s12*x_q/4*(1+cos(theta))",
          "t14" : "-2*s12*x_qbar/4*(1-(((2-x_qbar-x_q)**2-x_qbar**2-x_q**2)/(2*x_q*x_qbar))*cos(theta)+sqrt(1-(((2-x_qbar-x_q)**2-x_qbar**2-x_q**2)/(2*x_q*x_qbar))**2)*sin(theta)*sin(eta))",
          "t24" : "-2*s12*x_qbar/4*(1+(((2-x_qbar-x_q)**2-x_qbar**2-x_q**2)/(2*x_q*x_qbar))*cos(theta)-sqrt(1-(((2-x_qbar-x_q)**2-x_qbar**2-x_q**2)/(2*x_q*x_qbar))**2)*sin(theta)*sin(eta))",
          "s34" : "2*s12/4*x_q*x_qbar * (1-(((2-x_qbar-x_q)**2-x_qbar**2-x_q**2)/(2*x_q*x_qbar)))",
         }).simplify()
sret

128*G**2*ee**4*(-x_q**4*cos(theta)**2 - x_q**4 + x_q**2*x_qbar**2*sqrt((x_q**2*x_qbar - x_q**2 + x_q*x_qbar**2 - 3*x_q*x_qbar + 2*x_q - x_qbar**2 + 2*x_qbar - 1)/(x_q**2*x_qbar**2))*(cos(eta - 2*theta) - cos(eta + 2*theta)) - x_q**2*x_qbar**2*cos(theta)**2 - x_q**2*x_qbar**2 - 2*x_q**2*x_qbar*sqrt((x_q**2*x_qbar - x_q**2 + x_q*x_qbar**2 - 3*x_q*x_qbar + 2*x_q - x_qbar**2 + 2*x_qbar - 1)/(x_q**2*x_qbar**2))*(cos(eta - 2*theta) - cos(eta + 2*theta)) - 4*x_q**2*x_qbar*sin(eta)**2*sin(theta)**2 + 4*x_q**2*x_qbar*cos(theta)**2 + 4*x_q**2*sin(eta)**2*sin(theta)**2 - 4*x_q**2*cos(theta)**2 - 2*x_q*x_qbar**2*sqrt((x_q**2*x_qbar - x_q**2 + x_q*x_qbar**2 - 3*x_q*x_qbar + 2*x_q - x_qbar**2 + 2*x_qbar - 1)/(x_q**2*x_qbar**2))*(cos(eta - 2*theta) - cos(eta + 2*theta)) - 4*x_q*x_qbar**2*sin(eta)**2*sin(theta)**2 + 4*x_q*x_qbar**2*cos(theta)**2 + 2*x_q*x_qbar*sqrt((x_q**2*x_qbar - x_q**2 + x_q*x_qbar**2 - 3*x_q*x_qbar + 2*x_q - x_qbar**2 + 2*x_qbar - 1)/(x_q**2*x_qbar**2))*(cos(eta - 2*theta) - cos(e

In [18]:
import sympy
int1 = sympy.integrate(sret, ("eta",0,2*sympy.pi))
int2 = sympy.integrate(int1 * sympy.parse_expr("sin(theta)"), ("theta",0,sympy.pi))
int3 = sympy.integrate(int2, ("phi",0,2*sympy.pi))
int3

2*pi*(-2048*pi*G**2*ee**4*x_q**4/(27*s12*x_q**3*x_qbar - 27*s12*x_q**3 - 27*s12*x_q**2*x_qbar + 27*s12*x_q**2) - 2048*pi*G**2*ee**4*x_q**2*x_qbar**2/(27*s12*x_q**3*x_qbar - 27*s12*x_q**3 - 27*s12*x_q**2*x_qbar + 27*s12*x_q**2))

In [19]:
ssret = int3.simplify()
ssret

-4096*pi**2*G**2*ee**4*(x_q**2 + x_qbar**2)/(27*s12*(x_q*x_qbar - x_q - x_qbar + 1))

In [20]:
import feynamp
feynamp.get_color_average(fds)

['1', '1']

In [21]:
feynamp.get_spin_average(fds)

['1/2', '1/2']